In [1]:
import csv
import numpy as np
import pandas as pd

In [2]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [3]:
import xgboost as xgb
from sklearn.svm import SVR, LinearSVR
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, Lasso

In [4]:
f = pd.read_csv('C:\\beijing_engineered.csv')
df = pd.DataFrame(f)

In [5]:
df

,pm25,dewp,temp,cws,target,cbwd_1,cbwd_2,cbwd_3,clunar,cstorm,...,chour_16.0,chour_17.0,chour_18.0,chour_19.0,chour_20.0,chour_21.0,chour_22.0,chour_23.0,pm25_1,pm25_2
0,159,-11,-5,3,181,0,0,0,0,0,...,0,0,0,0,0,0,0,0,148,129
1,181,-7,-5,5,138,0,0,0,0,0,...,0,0,0,0,0,0,0,0,159,148
2,138,-7,-5,6,109,0,0,0,0,0,...,0,0,0,0,0,0,0,0,181,159
3,109,-7,-6,7,105,0,0,0,0,0,...,0,0,0,0,0,0,0,0,138,181
4,105,-7,-6,8,124,0,0,0,0,0,...,0,0,0,0,0,0,0,0,109,138
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43792,10,-22,-2,226,8,0,1,0,0,0,...,0,0,0,1,0,0,0,0,9,8
43793,8,-23,-2,231,10,0,1,0,0,0,...,0,0,0,0,1,0,0,0,10,9
43794,10,-22,-3,237,10,0,1,0,0,0,...,0,0,0,0,0,1,0,0,8,10
43795,10,-22,-3,242,8,0,1,0,0,0,...,0,0,0,0,0,0,1,0,10,8


In [6]:
df['date'] = pd.to_datetime(df.date)

In [7]:
# Setting DateTime index
df['date'] = pd.to_datetime(df.date)
df.set_index('date', inplace=True)

In [8]:
df[['pm25', 'target']].describe()

,pm25,target
count,43797.000000,43797.000000
mean,98.025367,98.022011
std,90.522403,90.522867
min,0.000000,0.000000
25%,30.000000,30.000000
50%,74.000000,74.000000
75%,134.000000,134.000000
max,994.000000,994.000000


# Subsetting Data for Cross-Validation

The cross-validation procedure requires that the train-validation data (ex holdout test set) be subsetted into **five time-ordered portions** for CV=5, using the **forward chain approach**.

Each of the five folds are then split into **three time-ordered portions** of which the final portion is the **validation set**, which is of a **fixed length**.

In [9]:
# Exclude the last 10% of observations, which is the holdout-test set that will not be used in the CV
X = df.iloc[:-4380]

In [10]:
# Size of overall CV train-validation data
len(X)

39417

**Each fold** of the 5-folds of data are then **split 3-ways**:
- A **fixed validation set** sized at 10% of the overall CV data
- 2/3 of remaining fold for the base training data (earliest in time)
- 1/3 of remaining fold for the meta training data (later in time)

Given **CV=5**, that means 5 X validation sample lengths should be set ahead of the first training data (base+meta) fold

In [11]:
# Each fixed validation sub-sample size
n_valid = round(len(X)*0.1)
n_valid

3942

In [12]:
n_split = 0
splits_list = []

while n_split < len(X):
    if n_split == 0:
        n_split = len(X) - (n_valid*5) # At the start, set the first training fold without the validation set
    
    else:
        n_split += n_valid # Add the fixed validation set to each training fold
        splits_list.append(n_split)
    

In [13]:
# Index numbers for where each CV fold's splice ends
splits_list

[23649, 27591, 31533, 35475, 39417]

### Generating DataFrames of the Rolling Forward-Chained Folds

In [14]:
df_list = [] # List of the CV folds as DataFrames

for i in splits_list:
    df_list.append(X[:i])
    

In [15]:
len(df_list[0])

23649

In [16]:
# Confirming that the final split covers the full CV data
len(df_list[4]) - len(X)

0

# Post-Gridsearch Models

In [17]:
# Lasso Regression
las = Lasso(fit_intercept=1, alpha=0.05, max_iter=10000, random_state=8)

In [18]:
# Multi-Layer Perceptron
mlp = MLPRegressor(hidden_layer_sizes=(36,12), alpha=1e-05, activation='relu', early_stopping=True,
                   max_iter=20000, random_state=8)

In [19]:
# Linear Support Vector Regression
svr = LinearSVR(epsilon=11, C=37, fit_intercept=True, loss='squared_epsilon_insensitive', 
                max_iter=30000, random_state=8)

In [20]:
# Support Vector Machine
svm = SVR(kernel='poly', gamma=0.2, C=0.1, max_iter=30000)

In [21]:
# Random Forest Regressor
rf = RandomForestRegressor(n_estimators=375, min_samples_split=2, max_features='sqrt', random_state=8)

In [22]:
# XG Boost Regressor
xgb = xgb.XGBRegressor(n_estimators=100, gamma=230, eta=0.1, subsample=0.75, objective='reg:squarederror', 
                       random_state=8)

In [23]:
# Dictionary of above base models
base_dict = {
    'Lasso Regression': las,
    'Multi-Layer Perceptron': mlp, 
    'Support Vector Regression': svr,
    'Support Vector Machine': svm,
    'Random Forest': rf,
    'XG Boost': xgb
}

In [24]:
# Setting Linear Regression as the final stage meta model
meta = LinearRegression()

# Functions

In [25]:
scaler = StandardScaler()

In [26]:
def fold_scoring(df_fold, meta_model, base_dict):
    '''
    Cycle through a dictionary of base models, fitting them then making predictions. The predictions are also 
    used as inputs for the meta model. All model predictions on the validation set are scored according to 
    MAE & RMSE. The output is two DataFrames containing the all-model scores.
    
    df_fold: DataFrame containing the sub-sample fold of interest, with the target variable in the last column
    
    meta_model: Meta model
    
    base_dict: Dictionary of base models
    '''    
    
    # Empty scorer DataFrames
    df_mae = pd.DataFrame(columns=base_dict.keys()) # Columns of DF will accord with base_dict keys
    df_rmse = pd.DataFrame(columns=base_dict.keys())
    
    # Empty dictionaries to hold the base model predictions for the meta model
    meta_dict1 = {}
    meta_dict2 = {}
    
    # Split the data length into 2/3 base-training data and 1/3 meta-training data
    length = len(df_fold) - n_valid # n_valid is given as per above
    n_meta = round(length/3)
    n_base = n_meta * 2
    
    X_base = df_fold.iloc[:n_base,:]
    X_meta = df_fold.iloc[n_base:(n_base + n_meta),:]
    X_valid = df_fold.iloc[length:(length + n_valid),:]
    
    # Extract target variables from each sub-sample
    y_meta = X_meta.pop('target')
    y_base = X_base.pop('target')
    y_valid = X_valid.pop('target')
    
    # Scaling the sub-samples
    X_bsc = scaler.fit_transform(X_base)
    X_msc = scaler.transform(X_meta)
    X_vsc = scaler.transform(X_valid)
    
    # Fit the base models to the base-training set, and generate predictions
    for key, reg in base_dict.items():
        reg.fit(X_bsc, y_base)
        
        z_meta = reg.predict(X_msc).tolist() # Generate predictions on meta-training set
        meta_dict1[key] = z_meta # Append predictions to dictionary
        
        z_valid = reg.predict(X_vsc).tolist() # Generate predictions on validation set for meta model
        meta_dict2[key] = z_valid

        df_mae[key] = pd.Series(round(mean_absolute_error(z_valid, y_valid), 4)) # Generate MAE scores for base models
        df_rmse[key] = pd.Series(round(mean_squared_error(z_valid, y_valid, squared=False), 4)) # Generate RMSE scores
    
    # Transform dictionary of predictions for meta model into DataFrames        
    df_meta1 = pd.DataFrame.from_dict(meta_dict1)
    df_meta2 = pd.DataFrame.from_dict(meta_dict2)
    
    # Train meta model using base models' predictions of meta-training set
    meta_model.fit(df_meta1, y_meta)
    
    # Generate meta model predictions of validation set
    meta_y = meta_model.predict(df_meta2)            
    
    # Obtain scores from meta model, and insert into respective scorer DataFrames
    df_mae['Stack Model'] = round(mean_absolute_error(meta_y, y_valid), 4)
    df_rmse['Stack Model'] = round(mean_squared_error(meta_y, y_valid, squared=False), 4)

    return df_mae, df_rmse


In [27]:
def stack_scores(df1, df2, df3, df4, df5):
    '''
    Objective: Stack the scorer DataFrames, plus calculate mean, median and standard deviation of scores

    '''    
    new_index = ['1st fold', '2nd fold', '3rd fold', '4th fold', '5th fold']
    
    df_new = pd.concat([df1, df2, df3, df4, df5])
    df_new.index = new_index
    
    x = len(df_new)
    
    df_new.loc['mean'] = round(df_new.mean(), 4)
    df_new.loc['median'] = round(df_new.iloc[:x,:].median(), 4)
    df_new.loc['stdev'] = round(df_new.iloc[:x,:].std(), 4)
    
    return df_new
    

# Scoring Each Fold

In [28]:
mae_1st, rmse_1st = fold_scoring(df_list[0], meta, base_dict)

C:\Users\Tan\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\Tan\anaconda3\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=30000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'


In [29]:
mae_1st

,Lasso Regression,Multi-Layer Perceptron,Support Vector Regression,Support Vector Machine,Random Forest,XG Boost,Stack Model
0,11.2777,12.5906,12.2504,13.483,13.2545,11.4165,11.0952


In [30]:
rmse_1st

,Lasso Regression,Multi-Layer Perceptron,Support Vector Regression,Support Vector Machine,Random Forest,XG Boost,Stack Model
0,18.9254,19.7555,19.4904,20.9506,19.9403,18.7335,18.5377


In [31]:
mae_2nd, rmse_2nd = fold_scoring(df_list[1], meta, base_dict)

C:\Users\Tan\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\Tan\anaconda3\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=30000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'


In [32]:
mae_2nd

,Lasso Regression,Multi-Layer Perceptron,Support Vector Regression,Support Vector Machine,Random Forest,XG Boost,Stack Model
0,14.3973,15.2224,15.2739,18.9752,17.787,16.3779,14.7688


In [33]:
rmse_2nd

,Lasso Regression,Multi-Layer Perceptron,Support Vector Regression,Support Vector Machine,Random Forest,XG Boost,Stack Model
0,27.762,27.896,28.0028,36.2795,36.8735,40.5363,29.2265


In [34]:
mae_3rd, rmse_3rd = fold_scoring(df_list[2], meta, base_dict)

C:\Users\Tan\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\Tan\anaconda3\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=30000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'


In [35]:
mae_3rd

,Lasso Regression,Multi-Layer Perceptron,Support Vector Regression,Support Vector Machine,Random Forest,XG Boost,Stack Model
0,11.719,12.4461,12.3813,13.716,13.4982,11.6168,11.5525


In [36]:
rmse_3rd

,Lasso Regression,Multi-Layer Perceptron,Support Vector Regression,Support Vector Machine,Random Forest,XG Boost,Stack Model
0,20.6169,20.8541,20.8621,23.0835,22.6239,20.8051,20.4243


In [37]:
mae_4th, rmse_4th = fold_scoring(df_list[3], meta, base_dict)

C:\Users\Tan\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\Tan\anaconda3\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=30000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'


In [38]:
mae_4th

,Lasso Regression,Multi-Layer Perceptron,Support Vector Regression,Support Vector Machine,Random Forest,XG Boost,Stack Model
0,12.7882,13.6052,13.4848,15.2743,14.7573,12.8028,12.7065


In [39]:
rmse_4th

,Lasso Regression,Multi-Layer Perceptron,Support Vector Regression,Support Vector Machine,Random Forest,XG Boost,Stack Model
0,21.8869,22.3795,22.0211,25.0217,25.5284,22.7118,21.8517


In [40]:
mae_5th, rmse_5th = fold_scoring(df_list[4], meta, base_dict)

C:\Users\Tan\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\Tan\anaconda3\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=30000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'


In [41]:
mae_5th

,Lasso Regression,Multi-Layer Perceptron,Support Vector Regression,Support Vector Machine,Random Forest,XG Boost,Stack Model
0,12.3165,12.802,13.3891,14.5751,13.3869,11.8918,12.118


In [42]:
rmse_5th

,Lasso Regression,Multi-Layer Perceptron,Support Vector Regression,Support Vector Machine,Random Forest,XG Boost,Stack Model
0,23.1006,22.9184,25.6771,40.0732,23.107,22.5064,22.9992


# Comparative CV Scores

Putting all the scores of the various folds together.

In [43]:
stack_scores(mae_1st, mae_2nd, mae_3rd, mae_4th, mae_5th)

,Lasso Regression,Multi-Layer Perceptron,Support Vector Regression,Support Vector Machine,Random Forest,XG Boost,Stack Model
1st fold,11.2777,12.5906,12.2504,13.4830,13.2545,11.4165,11.0952
2nd fold,14.3973,15.2224,15.2739,18.9752,17.7870,16.3779,14.7688
3rd fold,11.7190,12.4461,12.3813,13.7160,13.4982,11.6168,11.5525
4th fold,12.7882,13.6052,13.4848,15.2743,14.7573,12.8028,12.7065
5th fold,12.3165,12.8020,13.3891,14.5751,13.3869,11.8918,12.1180
mean,12.4997,13.3333,13.3559,15.2047,14.5368,12.8212,12.4482
median,12.3165,12.8020,13.3891,14.5751,13.4982,11.8918,12.1180
stdev,1.2063,1.1471,1.2112,2.2248,1.9143,2.0578,1.4312


In [44]:
stack_scores(rmse_1st, rmse_2nd, rmse_3rd, rmse_4th, rmse_5th)

,Lasso Regression,Multi-Layer Perceptron,Support Vector Regression,Support Vector Machine,Random Forest,XG Boost,Stack Model
1st fold,18.9254,19.7555,19.4904,20.9506,19.9403,18.7335,18.5377
2nd fold,27.7620,27.8960,28.0028,36.2795,36.8735,40.5363,29.2265
3rd fold,20.6169,20.8541,20.8621,23.0835,22.6239,20.8051,20.4243
4th fold,21.8869,22.3795,22.0211,25.0217,25.5284,22.7118,21.8517
5th fold,23.1006,22.9184,25.6771,40.0732,23.1070,22.5064,22.9992
mean,22.4584,22.7607,23.2107,29.0817,25.6146,25.0586,22.6079
median,21.8869,22.3795,22.0211,25.0217,23.1070,22.5064,21.8517
stdev,3.3444,3.1308,3.5291,8.5323,6.5992,8.7989,4.0579


The Stack Model has the **lowest mean and median MAE scores**, with Lasso Regression coming in second.

The Stack Model has the **second lowest mean RMSE score**, behind Lasso Regression, but the **lowest median RMSE score**.